In [1]:
# 필요한 라이브러리가 설치되어 있지 않다면 주석을 해제하고 실행하세요.
# !pip install opencv-python pandas openpyxl

import cv2
import pandas as pd
import os
import subprocess

print(f"OpenCV version: {cv2.__version__}")
print(f"Pandas version: {pd.__version__}")

OpenCV version: 4.5.3
Pandas version: 1.4.2


In [2]:
# --- 사용자 설정 ---
# 샘플 영상 파일 이름
video_file = './sample data/P01_R01.mp4'
# GT 정보가 담긴 Excel 파일 이름
excel_file = './sample data/Action-Meta-action-list.xlsx'
# 분할된 영상들을 저장할 디렉토리 이름
output_dir = 'split_videos'

# --- 설정 확인 ---
# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' 디렉토리를 생성했습니다.")

# 파일 존재 여부 확인
if not os.path.exists(video_file):
    print(f"[오류] 영상 파일 '{video_file}'을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
if not os.path.exists(excel_file):
    print(f"[오류] Excel 파일 '{excel_file}'을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

In [4]:
# --- 2. FFmpeg 명령어로 영상 분할 실행 ---
print("FFmpeg를 사용하여 영상 분할을 시작합니다...")

### --- 수정된 부분 --- ###
# 경로를 포함한 파일명에서 순수 파일명(확장자 제외)만 정확히 추출
filename_only = os.path.basename(video_file)
base_name = os.path.splitext(filename_only)[0]
### ------------------ ###

# 각 뷰(V1~V4)에 대한 파일 경로 설정
output_v1 = os.path.join(output_dir, f'{base_name}_V1.mp4')
output_v2 = os.path.join(output_dir, f'{base_name}_V2.mp4')
output_v3 = os.path.join(output_dir, f'{base_name}_V3.mp4')
output_v4 = os.path.join(output_dir, f'{base_name}_V4.mp4')

command = [
    'ffmpeg', '-y', '-i', video_file,
    '-filter_complex',
    "[0:v]crop=iw/2:ih/2:0:0[tl];" +
    "[0:v]crop=iw/2:ih/2:iw/2:0[tr];" +
    "[0:v]crop=iw/2:ih/2:0:ih/2[bl];" +
    "[0:v]crop=iw/2:ih/2:iw/2:ih/2[br]",
    '-map', '[tl]', output_v1,
    '-map', '[tr]', output_v2,
    '-map', '[bl]', output_v3,
    '-map', '[br]', output_v4
]

try:
    subprocess.run(command, check=True, capture_output=True, text=True)
    print("영상 분할이 성공적으로 완료되었습니다!")
    print(f"-> 결과는 '{output_dir}' 폴더에 저장되었습니다.")
except FileNotFoundError:
    print("[오류] FFmpeg가 설치되어 있지 않거나 경로가 설정되지 않았습니다.")
except subprocess.CalledProcessError as e:
    print(f"[오류] FFmpeg 실행 중 오류가 발생했습니다.")
    print(f"-> FFmpeg 에러 로그: {e.stderr}") # 더 자세한 에러 로그 출력

FFmpeg를 사용하여 영상 분할을 시작합니다...
영상 분할이 성공적으로 완료되었습니다!
-> 결과는 'split_videos' 폴더에 저장되었습니다.


In [7]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

def parse_anvil_track(anvil_file, track_name):
    """
    지정된 .anvil 파일에서 특정 트랙 이름의 주석(annotation)을 파싱하여
    Pandas DataFrame으로 반환하는 함수.

    :param anvil_file: 파싱할 .anvil 파일 경로
    :param track_name: 추출할 트랙의 이름 (예: "Action Label")
    :return: 파싱된 데이터가 담긴 DataFrame
    """
    if not os.path.exists(anvil_file):
        print(f"[오류] Anvil 파일 '{anvil_file}'을 찾을 수 없습니다.")
        return None

    try:
        tree = ET.parse(anvil_file)
        root = tree.getroot()

        annotations = []
        
        # XML 파일 내에서 정확한 트랙 이름으로 해당 트랙을 찾음
        target_track = root.find(f'.//track[@name="{track_name}"]')

        if target_track is None:
            print(f"[정보] '{track_name}' 트랙을 찾을 수 없습니다.")
            return None

        # 트랙 내의 모든 'el' 태그를 순회
        for element in target_track.findall('el'):
            start_time = element.get('start')
            end_time = element.get('end')
            
            # 'el' 태그 내부의 'attribute' 태그에서 레이블 텍스트를 추출
            attribute = element.find('attribute')
            if attribute is not None and attribute.text:
                action_label = attribute.text
                annotations.append({
                    'Label': action_label,
                    'Start Time (s)': float(start_time),
                    'End Time (s)': float(end_time)
                })

        return pd.DataFrame(annotations)

    except ET.ParseError as e:
        print(f"[오류] XML 파일을 파싱하는 중 오류가 발생했습니다: {e}")
        return None
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")
        return None

# --- 사용자 설정 ---
anvil_file = './sample data/P01_R01.anvil' 

# --- "Action Label" 트랙 파싱 및 출력 ---
print(f"--- {anvil_file} | Track: Action Label ---")
action_df = parse_anvil_track(anvil_file, "Action Label")

if action_df is not None and not action_df.empty:
    # Jupyter Notebook 환경에서는 display를 사용하면 더 깔끔하게 보입니다.
    try:
        display(action_df)
    except NameError:
        print(action_df)

print("\n" + "="*50 + "\n")

# --- "Meta-Action Label" 트랙 파싱 및 출력 ---
print(f"--- {anvil_file} | Track: Meta-Action Label ---")
meta_action_df = parse_anvil_track(anvil_file, "Meta-Action Label")

if meta_action_df is not None and not meta_action_df.empty:
    try:
        display(meta_action_df)
    except NameError:
        print(meta_action_df)

--- ./sample data/P01_R01.anvil | Track: Action Label ---


,Label,Start Time (s),End Time (s)
0,[OP010] Consult sheets,2.72000,9.00000
1,[OP010] Catch Fixture key LARD,11.28000,12.84000
2,[OP010] Place LARD on Profile P360-1,13.84000,18.88000
3,[OP010] Catch Fixation FIXA1,23.32000,25.44000
4,[OP010] Place FIXA1 on LARD at 160mm,26.48000,28.80000
...,...,...,...
104,[OP070] Place ECR8 on DI2T,492.88000,497.88000
105,[OP070] Take screwdriver,504.64001,506.16000
106,[OP070] Screw P360-2 with B835,506.39999,509.79999
107,[OP070] Put down screwdriver,509.88000,512.08002




--- ./sample data/P01_R01.anvil | Track: Meta-Action Label ---


,Label,Start Time (s),End Time (s)
0,[2] Consult sheets,2.72000,9.00000
1,[7] Picking left,11.28000,12.84000
2,[12] Assemble system,13.84000,18.88000
3,[6] Picking in front,23.32000,25.44000
4,[12] Assemble system,26.48000,28.80000
...,...,...,...
104,[12] Assemble system,492.88000,497.88000
105,[4] Take screwdriver,504.64001,506.16000
106,[12] Assemble system,506.39999,509.79999
107,[5] Put down screwdriver,509.88000,512.08002


In [ ]:
import os
import subprocess

# --- 사용자 설정 ---
input_dir = 'split_videos'
output_dir = 'trimmed_videos' # 새 출력 폴더 지정
trim_duration = 48

# --- 폴더 생성 및 파일 확인 ---
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' 디렉토리를 생성했습니다.")

if not os.path.exists(input_dir):
    print(f"[오류] 입력 폴더 '{input_dir}'를 찾을 수 없습니다.")
    exit()

video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp4', '.avi'))]
if not video_files:
    print(f"[오류] '{input_dir}' 폴더에 처리할 영상 파일이 없습니다.")
    exit()

print(f"총 {len(video_files)}개의 영상을 재인코딩하여 {trim_duration}초로 자릅니다. (시간이 더 걸릴 수 있습니다)")
print("-" * 30)

# --- FFmpeg로 각 영상 자르기 (재인코딩) ---
for filename in video_files:
    input_path = os.path.join(input_dir, filename)
    base, ext = os.path.splitext(filename)
    output_filename = f"{base}_trimmed{ext}"
    output_path = os.path.join(output_dir, output_filename)
    
    print(f"Processing: {filename} -> {output_filename}")

    # -c copy 옵션을 제거하여 재인코딩 수행
    command = [
        'ffmpeg',
        '-y',
        '-i', input_path,
        '-ss', '0',
        '-t', str(trim_duration),
        # '-c', 'copy', # 이 부분을 제거하거나 주석 처리
        output_path
    ]

    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
    except FileNotFoundError:
        print("[오류] FFmpeg가 설치되어 있지 않거나 경로가 설정되지 않았습니다.")
        exit()
    except subprocess.CalledProcessError as e:
        print(f"[오류] {filename} 처리 중 FFmpeg 오류 발생:\n-> {e.stderr}")
        continue

print("-" * 30)
print("모든 영상 자르기 작업이 완료되었습니다!")
print(f"-> 결과는 '{output_dir}' 폴더에 저장되었습니다.")

'trimmed_videos_re-encoded' 디렉토리를 생성했습니다.
총 4개의 영상을 재인코딩하여 48초로 자릅니다. (시간이 더 걸릴 수 있습니다)
------------------------------
Processing: P01_R01_V1.mp4 -> P01_R01_V1_trimmed.mp4
Processing: P01_R01_V2.mp4 -> P01_R01_V2_trimmed.mp4
Processing: P01_R01_V4.mp4 -> P01_R01_V4_trimmed.mp4
Processing: P01_R01_V3.mp4 -> P01_R01_V3_trimmed.mp4
------------------------------
모든 영상 자르기 작업이 완료되었습니다!
-> 결과는 'trimmed_videos_re-encoded' 폴더에 저장되었습니다.
